In [88]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#pos = pd.read_csv("/Users/hungduong/Documents/MSBA 2nd/IS 6812/home-credit-default-risk/POS_CASH_balance.csv")
#application_and_previous = pd.read_csv("/Users/hungduong/Documents/MSBA 2nd/IS 6812/home-credit-default-risk/application_and_previous.csv")
#previous_cleaned = pd.read_csv("/Users/hungduong/Documents/MSBA 2nd/IS 6812/home-credit-default-risk/previous_cleaned.csv")

# POS

### High view

In [91]:
print(pos.head())

   SK_ID_PREV  SK_ID_CURR  MONTHS_BALANCE  CNT_INSTALMENT  \
0     1803195      182943             -31            48.0   
1     1715348      367990             -33            36.0   
2     1784872      397406             -32            12.0   
3     1903291      269225             -35            48.0   
4     2341044      334279             -35            36.0   

   CNT_INSTALMENT_FUTURE NAME_CONTRACT_STATUS  SK_DPD  SK_DPD_DEF  
0                   45.0               Active       0           0  
1                   35.0               Active       0           0  
2                    9.0               Active       0           0  
3                   42.0               Active       0           0  
4                   35.0               Active       0           0  


In [92]:
print(pos.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001358 entries, 0 to 10001357
Data columns (total 8 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   SK_ID_PREV             int64  
 1   SK_ID_CURR             int64  
 2   MONTHS_BALANCE         int64  
 3   CNT_INSTALMENT         float64
 4   CNT_INSTALMENT_FUTURE  float64
 5   NAME_CONTRACT_STATUS   object 
 6   SK_DPD                 int64  
 7   SK_DPD_DEF             int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 610.4+ MB
None


In [93]:
pos.isna().mean()*100

SK_ID_PREV               0.000000
SK_ID_CURR               0.000000
MONTHS_BALANCE           0.000000
CNT_INSTALMENT           0.260675
CNT_INSTALMENT_FUTURE    0.260835
NAME_CONTRACT_STATUS     0.000000
SK_DPD                   0.000000
SK_DPD_DEF               0.000000
dtype: float64

### "CNT_INSTALMENT", "CNT_INSTALMENT_FUTURE"

Is there a systematic missingness? Could be with "NAME_CONTRACT_STATUS".

In [96]:
#Systematic missngness?
pos.groupby("NAME_CONTRACT_STATUS")[["CNT_INSTALMENT", "CNT_INSTALMENT_FUTURE"]].apply(lambda x: x.isna().sum())

,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE
NAME_CONTRACT_STATUS,,
Active,26,26
Amortized debt,0,0
Approved,2804,2794
Canceled,12,12
Completed,0,0
Demand,0,0
Returned to the store,2961,2962
Signed,20266,20291
XNA,2,2


There could be better imputation for the future, now, I will fillna with 0 since the missing percentage is low for both columns (0.26%)

In [98]:
pos[["CNT_INSTALMENT", "CNT_INSTALMENT_FUTURE"]]=pos[["CNT_INSTALMENT", "CNT_INSTALMENT_FUTURE"]].fillna(0)
pos.isna().mean()*100

SK_ID_PREV               0.0
SK_ID_CURR               0.0
MONTHS_BALANCE           0.0
CNT_INSTALMENT           0.0
CNT_INSTALMENT_FUTURE    0.0
NAME_CONTRACT_STATUS     0.0
SK_DPD                   0.0
SK_DPD_DEF               0.0
dtype: float64

### Join with previous_cleaned

Joining the CLEANED not aggregated data of previous application with pos.

In [125]:
#Joining
previous_pos = previous_cleaned.merge(pos, on=["SK_ID_PREV", "SK_ID_CURR"], how="left")


In [127]:
previous_pos.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS_y,SK_DPD,SK_DPD_DEF
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,35,Connectivity,12.0,middle,-1.0,1.0,0.0,Completed,0.0,0.0
1,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,35,Connectivity,12.0,middle,-2.0,12.0,12.0,Active,0.0,0.0
2,2802425,108129,Cash loans,25188.615,607500.0,679671.0,0.0,607500.0,THURSDAY,11,...,-1,XNA,36.0,low_action,-4.0,36.0,34.0,Active,0.0,0.0
3,2802425,108129,Cash loans,25188.615,607500.0,679671.0,0.0,607500.0,THURSDAY,11,...,-1,XNA,36.0,low_action,-5.0,36.0,35.0,Active,0.0,0.0
4,2802425,108129,Cash loans,25188.615,607500.0,679671.0,0.0,607500.0,THURSDAY,11,...,-1,XNA,36.0,low_action,-3.0,36.0,33.0,Active,0.0,0.0


In [131]:
previous_pos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10431762 entries, 0 to 10431761
Data columns (total 34 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   SK_ID_PREV                   int64  
 1   SK_ID_CURR                   int64  
 2   NAME_CONTRACT_TYPE           object 
 3   AMT_ANNUITY                  float64
 4   AMT_APPLICATION              float64
 5   AMT_CREDIT                   float64
 6   AMT_DOWN_PAYMENT             float64
 7   AMT_GOODS_PRICE              float64
 8   WEEKDAY_APPR_PROCESS_START   object 
 9   HOUR_APPR_PROCESS_START      int64  
 10  FLAG_LAST_APPL_PER_CONTRACT  object 
 11  NFLAG_LAST_APPL_IN_DAY       int64  
 12  RATE_DOWN_PAYMENT            float64
 13  NAME_CASH_LOAN_PURPOSE       object 
 14  NAME_CONTRACT_STATUS_x       object 
 15  DAYS_DECISION                int64  
 16  NAME_PAYMENT_TYPE            object 
 17  CODE_REJECT_REASON           object 
 18  NAME_TYPE_SUITE              object 
 19

### Aggregate and join with application_and_previous data frame

In [133]:
import pandas as pd

# Explicitly convert numeric columns
numeric_cols = [
    "AMT_ANNUITY", 
    "AMT_GOODS_PRICE", 
    "CNT_PAYMENT",
    "CNT_INSTALMENT",
    "CNT_INSTALMENT_FUTURE",
    "SK_DPD",
    "SK_DPD_DEF"
]

for col in numeric_cols:
    previous_pos[col] = pd.to_numeric(previous_pos[col], errors='coerce')

# Combined aggregation dictionary
agg_dict = {
    "SK_ID_PREV": "count",
    "AMT_ANNUITY": ["median", "sum"],
    "AMT_GOODS_PRICE": ["median", "sum"],
    "CNT_PAYMENT": "median",
    "CNT_INSTALMENT": "median",
    "CNT_INSTALMENT_FUTURE": "median",
    "SK_DPD": ["median", "max"],
    "SK_DPD_DEF": ["median", "max"],
    "DAYS_DECISION": "median",

    # Most common (mode) aggregation for categorical columns
    "NAME_CONTRACT_TYPE": lambda x: x.mode().iloc[0],
    "NAME_CASH_LOAN_PURPOSE": lambda x: x.mode().iloc[0],
    "NAME_PAYMENT_TYPE": lambda x: x.mode().iloc[0],
    "NAME_CONTRACT_STATUS_x": lambda x: x.mode().iloc[0]
}

# Perform aggregation
previous_pos_agg = previous_pos.groupby("SK_ID_CURR").agg(agg_dict).reset_index()

# Flatten MultiIndex columns
previous_pos_agg.columns = ['_'.join(col).strip('_') if isinstance(col, tuple) else col for col in previous_pos_agg.columns.values]

# Fill NaNs for numeric columns with 0
cols_to_fill = [col for col in previous_pos_agg.columns if 'median' in col or 'sum' in col or 'max' in col]
previous_pos_agg[cols_to_fill] = previous_pos_agg[cols_to_fill].fillna(0)

print(previous_pos_agg.head())

   SK_ID_CURR  SK_ID_PREV_count  AMT_ANNUITY_median  AMT_ANNUITY_sum  \
0      100001                 5            3951.000        19755.000   
1      100002                19            9251.775       175783.725   
2      100003                28           64567.665      1384245.000   
3      100004                 4            5357.250        21429.000   
4      100005                12            4813.200        52945.200   

   AMT_GOODS_PRICE_median  AMT_GOODS_PRICE_sum  CNT_PAYMENT_median  \
0                 24835.5             124177.5                 8.0   
1                179055.0            3402045.0                24.0   
2                337500.0           10725714.0                12.0   
3                 24282.0              97128.0                 4.0   
4                 44617.5             490792.5                12.0   

   CNT_INSTALMENT_median  CNT_INSTALMENT_FUTURE_median  SK_DPD_median  \
0                    4.0                           2.0            0.0   


In [135]:
previous_pos_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338857 entries, 0 to 338856
Data columns (total 18 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   SK_ID_CURR                       338857 non-null  int64  
 1   SK_ID_PREV_count                 338857 non-null  int64  
 2   AMT_ANNUITY_median               338857 non-null  float64
 3   AMT_ANNUITY_sum                  338857 non-null  float64
 4   AMT_GOODS_PRICE_median           338857 non-null  float64
 5   AMT_GOODS_PRICE_sum              338857 non-null  float64
 6   CNT_PAYMENT_median               338857 non-null  float64
 7   CNT_INSTALMENT_median            338857 non-null  float64
 8   CNT_INSTALMENT_FUTURE_median     338857 non-null  float64
 9   SK_DPD_median                    338857 non-null  float64
 10  SK_DPD_max                       338857 non-null  float64
 11  SK_DPD_DEF_median                338857 non-null  float64
 12  SK

### Join with the application_final

In [141]:
#load
application_final = pd.read_csv("/Users/hungduong/Documents/MSBA 2nd/IS 6812/home-credit-default-risk/application_final.csv")

In [143]:
previous_pos_application = application_final.merge(previous_pos_agg, on=["SK_ID_CURR"], how="left")


In [147]:
previous_pos_application.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CNT_INSTALMENT_FUTURE_median,SK_DPD_median,SK_DPD_max,SK_DPD_DEF_median,SK_DPD_DEF_max,DAYS_DECISION_median,NAME_CONTRACT_TYPE_<lambda>,NAME_CASH_LOAN_PURPOSE_<lambda>,NAME_PAYMENT_TYPE_<lambda>,NAME_CONTRACT_STATUS_x_<lambda>
0,343403,0,Cash loans,M,N,Y,-0.577538,-0.332309,-0.359080,-0.119574,...,12.0,0.0,0.0,0.0,0.0,-931.0,Consumer loans,XAP,Cash through the bank,Approved
1,239949,0,Cash loans,F,N,N,-0.577538,-0.408219,-0.752182,-0.761646,...,3.0,0.0,0.0,0.0,0.0,-939.0,Consumer loans,XAP,Cash through the bank,Approved
2,340073,0,Cash loans,F,N,N,-0.577538,-0.294354,-0.781699,-0.584051,...,4.5,0.0,0.0,0.0,0.0,-2248.0,Consumer loans,XAP,Cash through the bank,Approved
3,442204,0,Cash loans,F,N,Y,-0.577538,-0.237421,-0.722733,-0.708243,...,5.5,0.0,0.0,0.0,0.0,-300.0,Cash loans,XNA,Cash through the bank,Approved
4,443323,0,Cash loans,F,Y,Y,-0.577538,1.185892,-0.237918,2.065580,...,3.5,0.0,0.0,0.0,0.0,-1756.0,Consumer loans,XAP,XNA,Approved


In [155]:
previous_pos_application.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49650 entries, 0 to 49649
Data columns (total 62 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   SK_ID_CURR                       49650 non-null  int64  
 1   TARGET                           49650 non-null  int64  
 2   NAME_CONTRACT_TYPE               49650 non-null  object 
 3   CODE_GENDER                      49650 non-null  object 
 4   FLAG_OWN_CAR                     49650 non-null  object 
 5   FLAG_OWN_REALTY                  49650 non-null  object 
 6   CNT_CHILDREN                     49650 non-null  float64
 7   AMT_INCOME_TOTAL                 49650 non-null  float64
 8   AMT_CREDIT                       49650 non-null  float64
 9   AMT_ANNUITY                      49650 non-null  float64
 10  AMT_GOODS_PRICE                  49650 non-null  float64
 11  NAME_INCOME_TYPE                 49650 non-null  object 
 12  NAME_EDUCATION_TYP

In [157]:
previous_pos_application.isna().mean()*100

SK_ID_CURR                         0.000000
TARGET                             0.000000
NAME_CONTRACT_TYPE                 0.000000
CODE_GENDER                        0.000000
FLAG_OWN_CAR                       0.000000
                                     ...   
DAYS_DECISION_median               4.662638
NAME_CONTRACT_TYPE_<lambda>        4.662638
NAME_CASH_LOAN_PURPOSE_<lambda>    4.662638
NAME_PAYMENT_TYPE_<lambda>         4.662638
NAME_CONTRACT_STATUS_x_<lambda>    4.662638
Length: 62, dtype: float64

There is 4.66% missing from the aggregation, fillna for now?

In [160]:
#Write
previous_pos_application.to_csv("previous_pos_application.csv", index=False)